In [ ]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import os, gc
from sklearn.impute import SimpleImputer
import pickle
from dask import dataframe as dd
from dask import persist

In [ ]:
y = pd.read_csv('../input/amex-default-prediction/train_data.csv',nrows=5)
cols = y.columns
z = pd.read_csv('../input/amex-default-prediction/train_data.csv',nrows=5,skiprows=100,names=cols)
z

In [ ]:
z.groupby('customer_ID').mean()

In [ ]:
# change the structure of customer id to a int8 dtype to preserve RAM
def code_id(df):
    c_id = np.array(df['customer_ID'])
    new_id = []
    num = 0
    for i in range(len(c_id)):
        if i == 0:
            new_id.append(num)
            continue
            
        if c_id[i] == c_id[i-1]:
            new_id.append(num)
            
        if c_id[i] != c_id[i-1]:
            new_id.append(num+1)
            num+=1
            
    df['customer_ID'] = new_id
    df['customer_ID'] = df['customer_ID'].astype('int8')
    
    _ = gc.collect()
    
    return df  

In [ ]:
def clean_data(df):
    
    d_64 = np.array(df['D_64'])

    for i in range(len(d_64)):
        if d_64[i] == '':
            d_64[i] = 'X'
        
    df['D_64'] = d_64
    
    _ = gc.collect()
    cols = df.columns
    for j in range(len(cols)):
        if df[cols[j]].isna().sum() >= (df.shape[0] * 0.30):
            df = df.drop(cols[j],axis=1)
            
    # For some reason the ffill function does not work for dtypes float16 therefore they need to be converted to float64 dtypes
    _ = gc.collect()
    new_cols = df.columns
    for k in range(len(new_cols)):
        if df[new_cols[k]].dtype.name == 'float16':
            df[new_cols[k]] = df[new_cols[k]].astype('float64')
            
    df = df.ffill()
    df = df.bfill() 
    
    new_cols = df.columns
    for l in range(len(new_cols)):
        if df[new_cols[l]].dtype.name == 'float64':
            df[new_cols[l]] = df[new_cols[l]].astype('float16')
            
    _ = gc.collect()
    
    return df

In [ ]:
#nrows in training data = 5531451
#nrows in test data = 11363762
def read_data(path):
    
    df = pd.read_csv(path,nrows=1)
    cols=df.columns
    
    i=11
    skip=1
    while i > 0:
        
        try:
            train = pd.read_csv(path,nrows=500000,skiprows=skip,names=cols)
            train = clean_data(train)
            train = train.grouby('customer_ID').mean()
            
             
        except:
            train = pd.read_csv(path,skiprows=skip,names=cols)
            train = clean_data(train)
            train = train.grouby('customer_ID').mean()
            
            
        skip+=500000
        i-=1
        df.append([train],ignore_index=True)
        
    return df
    
    
    

In [ ]:
train_df = read_data('../input/amex-default-prediction/train_data.csv')

In [ ]:
train_df.to_feather('train_data')